<a href="https://colab.research.google.com/github/dulshanb/Programming.-/blob/master/Mela_Classifi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from tensorflow import keras
import tensorflow as tf
import numpy as np



In [0]:

import requests, zipfile, io

r = requests.get( 'https://github.com/dulshanb/Skin-detection-data/blob/master/mel_detect_processed.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


In [0]:
DIMEN = 32

X1 = np.load( 'processed_data/x1.npy')
X2 = np.load( 'processed_data/x2.npy')
Y = np.load( 'processed_data/y.npy')

X1 = X1.reshape( ( X1.shape[0]  , DIMEN**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , DIMEN**2 * 3  ) ).astype( np.float32 )

train_X1 = X1[ : 4500 ] 
train_X2 = X2[ : 4500 ] 
train_Y = Y[ : 4500 ] 

test_X1 = X1[ 4500 : ]
test_X2 = X2[ 4500 : ] 
test_Y = Y[ 4500 : ]
 
print(  train_X1.shape )
print(  train_X2.shape )
print(  train_Y.shape )

print(test_X1.shape )
print(test_X2.shape )
print(test_Y.shape) 


import pandas as pd 
pd.DataFrame(Y).iloc[:,0].value_counts()

In [0]:

import tensorflow.keras.backend as K

input_shape = ( (DIMEN**2) * 3 , )
convolution_shape = ( DIMEN , DIMEN , 3 )

kernel_size_1 = ( 8 , 8 )
kernel_size_2 = ( 6 , 6 )
kernel_size_3 = ( 4 , 4 )

pool_size_1 = ( 6 , 6 )
pool_size_2 = ( 4 , 4 )

strides = 1

seq_conv_model = [

	tf.keras.layers.Reshape( input_shape=input_shape , target_shape=convolution_shape),
	
	tf.keras.layers.Conv2D( 32, kernel_size=kernel_size_1 , strides=strides ,activation='relu' ),
	tf.keras.layers.MaxPooling2D(pool_size=pool_size_1, strides=strides ),
	tf.keras.layers.Dropout(0.3),
	
	tf.keras.layers.Conv2D( 64, kernel_size=kernel_size_2 , strides=strides ,activation='relu'),
	tf.keras.layers.MaxPooling2D(pool_size=pool_size_2 , strides=strides),
   tf.keras.layers.Dropout(0.3), 
    tf.keras.layers.Conv2D( 128, kernel_size=kernel_size_3 , strides=strides ,activation='relu'),
    
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dropout(0.3),
	tf.keras.layers.Dense( 3076 , activation=tf.keras.activations.sigmoid )

]

seq_model = tf.keras.Sequential( seq_conv_model )

input_x1 = tf.keras.layers.Input( shape=input_shape )
input_x2 = tf.keras.layers.Input( shape=input_shape )

output_x1 = seq_model( input_x1 )
output_x2 = seq_model( input_x2 )

distance_euclid = tf.keras.layers.Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
outputs = tf.keras.layers.Dense( 1 , activation=tf.keras.activations.sigmoid) ( distance_euclid )
model = tf.keras.models.Model( [ input_x1 , input_x2 ] , outputs )

model.compile( loss=tf.keras.losses.binary_crossentropy , optimizer=tf.keras.optimizers.Adam(lr=0.0001) , metrics=['accuracy'])

model.summary()


In [0]:
#Class Balance
import pandas as pd
pd.value_counts(train_Y.flatten())


In [0]:

model.fit( [ train_X1 , train_X2 ] , train_Y , epochs=15 , batch_size=24,validation_data=([ test_X1 , test_X2 ] , test_Y),shuffle=True ) 


In [0]:
#Saving model
model.save( 'model_melanoma.h5' )

In [0]:
from tensorflow.keras.models import load_model 
model=load_model('model_melanoma.h5')
metrics = model.evaluate( [ test_X1 , test_X2 ] , test_Y ) 
print( 'Loss of {} and Accuracy is {} %'.format( metrics[0] , metrics[1] * 100 ) ) 


In [0]:
y_pred = model.predict([ test_X1 , test_X2 ])
y_pred[y_pred<=.5]=0
y_pred[y_pred>.5]=1

y_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(test_Y, y_pred)

In [0]:

from sklearn.metrics import classification_report

target_names = ['y=0(Different Class)', 'y=1(Same Class)']
print(classification_report(test_Y, y_pred, target_names=target_names))

In [0]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(test_Y, y_pred)

In [0]:
precision

In [0]:
recall

In [0]:
thresholds

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) 
converter.post_training_quantize = True
tflite_model = converter.convert()
size = open( 'melanoma.tflite' , 'wb' ).write( tflite_model ) 
print( 'Model size > {} Megabytes'.format( size / 1024**2))